# Use examples of [premise](https://github.com/romainsacchi/premise) with user-generated scenarios

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment, using **user-generated scenarios**.

User-generated scenario are scenarios built by the premise users community. They can be used on their own
or together with a global IAM scenario.
Public community scenarios are listed under the repository [Premise community scenario](https://github.com/premise-community-scenarios).

User-generated scenarios are "packaged" into [data packages](https://specs.frictionlessdata.io/data-package/).

Data packages ensure that the data used comes with all necessary metadata, resources (scenario data, inventories, etc.), and that the data is formatted in a correct way.

To fetch a data packge, you can use the `datapackage` library.
For exmaple, let's fetch the user-generated scenario about [Switzerland's future energy supply](https://github.com/premise-community-scenarios/energy-perspective-2050-switzerland).

In [1]:
from datapackage import Package

In [2]:
# URL pointing to the raw datapackage.json file of the scenario
fp = "https://raw.githubusercontent.com/premise-community-scenarios/cobalt-perspective-2050/main/datapackage.json"
cobalt = Package(fp)

fp = r"https://raw.githubusercontent.com/premise-community-scenarios/ammonia-prospective-scenarios/main/datapackage.json"
ammonia = Package(fp)

#fp = "https://raw.githubusercontent.com/premise-community-scenarios/energy-perspective-2050-switzerland/main/datapackage.json"
fp = "/Users/romain/Library/CloudStorage/Dropbox/EP2050/energy-perspective-2050-switzerland/datapackage.json"
ep2050 = Package(fp)

fp = "https://raw.githubusercontent.com/premise-community-scenarios/hydrogen-prospective-scenarios/main/datapackage.json"
hydrogen = Package(fp)


In [3]:
hydrogen.resource_names

['scenario_data', 'inventories', 'config']

The datapackage has 3 resources: scenario data, inventories and a configuration file.

In [4]:
for resource in ep2050.resources:
    print(resource.name)

scenario_data
inventories
config


or

In [5]:
ammonia.resource_names

['scenario_data', 'config']

And you can directly read them (or look the resources up directly from the repo):

In [6]:
import pandas as pd
data = ammonia.get_resource("scenario_data").read()
pd.DataFrame(data).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,Sustainable development - image,CHN,Efficiency|Hydrogen|Alkaline Electrolysis (ele...,%,66,67.5,69,71,73,74.5,76,76
1,Sustainable development - image,INDIA,Efficiency|Hydrogen|Alkaline Electrolysis (ele...,%,66,67.5,69,71,73,74.5,76,76
2,Sustainable development - image,CAN,Efficiency|Hydrogen|Alkaline Electrolysis (ele...,%,66,67.5,69,71,73,74.5,76,76
3,Sustainable development - image,USA,Efficiency|Hydrogen|Alkaline Electrolysis (ele...,%,66,67.5,69,71,73,74.5,76,76
4,Sustainable development - image,MEX,Efficiency|Hydrogen|Alkaline Electrolysis (ele...,%,66,67.5,69,71,73,74.5,76,76


The datapackage can also be locally stored (i.e., on your computer), in which case you only
need to provide the filepath to the `datapackage.json` file.

Once the datapackage is loaded, you just need to pass it (in a list) to the `NewDatabase` instance of `premise`.

In [7]:
from premise import *
import bw2data
bw2data.projects.set_current("ecoinvent-3.10.1-cutoff")

In [8]:
external_scenarios = [
    #{"scenario": "Sustainable development", "data": cobalt},
    {"scenario": "Business As Usual - image", "data": ammonia},
    {"scenario": "ZERO Basis", "data": ep2050},
    #{"scenario": "Ambitious", "data": hydrogen},
]

scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2040, "external scenarios": external_scenarios},
    {"model": "image", "pathway":"SSP2-Base", "year": 2025, "external scenarios": external_scenarios},
    {"model": "image", "pathway":"SSP2-Base", "year": 2035, "external scenarios": external_scenarios},
    {"model": "image", "pathway":"SSP2-RCP26", "year": 2045, "external scenarios": external_scenarios},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent-3.10.1-cutoff",
        source_version="3.10",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)


premise v.(2, 2, 8)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

To integrate the projections of the user-generated scenario, call `update("external")` (or `ndb.update()` to apply all integrations, including that of external scenarios).

In [9]:
ndb.update("external")

Processing scenarios for sector 'external':   0%| | 0/4 [00:00<?, ?it/

Extracted 1 worksheets in 0.00 seconds
The following datasets will be duplicated:
+----------------------------------------------------+----------------------------------------------------+
|                        Name                        |                 Reference product                  |
+----------------------------------------------------+----------------------------------------------------+
| electricity production, hydro, reservoir, alpine r |             electricity, high voltage              |
| electricity production, nuclear, boiling water rea |             electricity, high voltage              |
| treatment of municipal solid waste, municipal inci | electricity, for reuse in municipal waste incinera |
| electricity production, natural gas, combined cycl |             electricity, high voltage              |
|        electricity production, photovoltaic        |              electricity, low voltage              |
| heat and power co-generation, wood chips, 6667 kW  |

Processing scenarios for sector 'external':  25%|▎| 1/4 [00:20<01:00, 

Extracted 1 worksheets in 0.00 seconds
The following datasets will be duplicated:
+----------------------------------------------------+----------------------------------------------------+
|                        Name                        |                 Reference product                  |
+----------------------------------------------------+----------------------------------------------------+
| electricity production, hydro, reservoir, alpine r |             electricity, high voltage              |
| electricity production, nuclear, boiling water rea |             electricity, high voltage              |
| treatment of municipal solid waste, municipal inci | electricity, for reuse in municipal waste incinera |
| electricity production, natural gas, combined cycl |             electricity, high voltage              |
|        electricity production, photovoltaic        |              electricity, low voltage              |
| heat and power co-generation, wood chips, 6667 kW  |

Processing scenarios for sector 'external':  50%|▌| 2/4 [00:42<00:43, 

Extracted 1 worksheets in 0.00 seconds
The following datasets will be duplicated:
+----------------------------------------------------+----------------------------------------------------+
|                        Name                        |                 Reference product                  |
+----------------------------------------------------+----------------------------------------------------+
| electricity production, hydro, reservoir, alpine r |             electricity, high voltage              |
| electricity production, nuclear, boiling water rea |             electricity, high voltage              |
| treatment of municipal solid waste, municipal inci | electricity, for reuse in municipal waste incinera |
| electricity production, natural gas, combined cycl |             electricity, high voltage              |
|        electricity production, photovoltaic        |              electricity, low voltage              |
| heat and power co-generation, wood chips, 6667 kW  |

Processing scenarios for sector 'external':  50%|▌| 2/4 [01:04<01:04, 


KeyboardInterrupt: 

In [ ]:
ndb.write_db_to_brightway()

You can combine the user-generated scenario with any, all or none of the projections for the IAM model.
For example, here with the electricity projections of the IMAGE SSP2-Base scenario:

In [ ]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2040},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='xxxxx',
        external_scenarios=[
            switzerland_2050,
        ]
)

ndb.update(
    [
        "electricity",
        "external"
    ]
)

Once the projections are integrated, you can export the database(s) back to your Brightway2 project, to
a CSV Simapro file, or as a set of sparse amtrices (see main exmaples notebook for more details).